In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow import keras
import numpy as np

batch_size = 64
epochs = 100
latent_dim = 256    #Latent dimensionality of encoding space
num_samples = 10000

data_path = r'fra-eng\fra.txt'

### Data Preparation

In [2]:
#Vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding = 'utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples,len(lines)-1)]:
    input_text, target_text, _ = line.split('\t')
    #we use tab as 'start sequence' character
    #for the targets and '\n' as 'end sequence' character
    target_text='\t'+target_text+'\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [3]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [4]:
print('Number of samples: ',len(input_texts))
print('Number of unique input tokens : ', num_encoder_tokens)
print('Number of unique output tokens : ', num_decoder_tokens)
print('Max sequence length for inputs : ', max_encoder_seq_length)
print('Max sequence length for outputs : ', max_decoder_seq_length)

Number of samples:  10000
Number of unique input tokens :  71
Number of unique output tokens :  93
Max sequence length for inputs :  15
Max sequence length for outputs :  59


In [5]:
input_token_index = dict([(char,i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i, char in enumerate(target_characters)])

### ONE HOT REPR

In [6]:
encoder_input_data = np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype = 'float64')
decoder_input_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens), dtype = 'float64')
decoder_target_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens), dtype = 'float64')

In [7]:
for i, (input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]]=1.
    encoder_input_data[i, t+1:, input_token_index[' ']]=1.
    for t, char in enumerate(target_text):
        #decoder target data is ahead of decoder input data by 1 timestamp
        decoder_input_data[i, t, target_token_index[char]]=1.
        if t > 0:
            #decoder target data will be ahead of 1 timestamp
            #and will not include the start character
            decoder_target_data[i, t-1, target_token_index[char]]=1.
        decoder_input_data[i, t+1:, target_token_index[' ']]=1.
        decoder_target_data[i, t:, target_token_index[' ']]=1.
         

In [8]:
encoder_input_data[0].shape

(15, 71)

### Building the model

In [9]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [10]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
# Define the model that will turn
# encoder_input_data & decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

### Training the Model

In [11]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 56s 375ms/step - loss: 2.1127 - val_loss: 2.3548
Epoch 2/100
125/125 [==============================] - 46s 367ms/step - loss: 2.1549 - val_loss: 2.5187
Epoch 3/100
125/125 [==============================] - 45s 359ms/step - loss: 2.2881 - val_loss: 2.6721
Epoch 4/100
125/125 [==============================] - 45s 363ms/step - loss: 2.4249 - val_loss: 2.8137
Epoch 5/100
125/125 [==============================] - 45s 359ms/step - loss: 2.5612 - val_loss: 2.9580
Epoch 6/100
125/125 [==============================] - 47s 372ms/step - loss: 2.6996 - val_loss: 3.1302
Epoch 7/100
125/125 [==============================] - 46s 372ms/step - loss: 2.8321 - val_loss: 3.3028
Epoch 8/100
125/125 [==============================] - 46s 370ms/step - loss: 2.9712 - val_loss: 3.4470
Epoch 9/100
125/125 [==============================] - 44s 353ms/step - loss: 3.1108 - val_loss: 3.5910
Epoch 10/100
125/125 [==============================] - 44s 353m

125/125 [==============================] - 34s 274ms/step - loss: 13.6451 - val_loss: 15.6747
Epoch 79/100
125/125 [==============================] - 37s 299ms/step - loss: 13.8106 - val_loss: 15.8775
Epoch 80/100
125/125 [==============================] - 36s 285ms/step - loss: 13.9771 - val_loss: 16.0487
Epoch 81/100
125/125 [==============================] - 35s 277ms/step - loss: 14.1390 - val_loss: 16.2516
Epoch 82/100
125/125 [==============================] - 34s 274ms/step - loss: 14.3079 - val_loss: 16.4492
Epoch 83/100
125/125 [==============================] - 34s 275ms/step - loss: 14.4737 - val_loss: 16.6229
Epoch 84/100
125/125 [==============================] - 36s 288ms/step - loss: 14.6369 - val_loss: 16.8316
Epoch 85/100
125/125 [==============================] - 35s 283ms/step - loss: 14.7991 - val_loss: 17.0134
Epoch 86/100
125/125 [==============================] - 34s 273ms/step - loss: 14.9677 - val_loss: 17.2024
Epoch 87/100
125/125 [============================


### INFERENCE SETUP


encode input and retrieve initial decoder state<br>
run one step of decoder with this initial state and a "start of sequence" token as target. Output will be the next target token.<br>
Repeat with the current target token and current states

In [20]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s.h5")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)


In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [13]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [19]:
# for seq_index in range(20):
#     # Take one sequence (part of the training set)
#     # for trying out decoding.
#     input_seq = encoder_input_data[seq_index : seq_index + 1]
#     decoded_sentence = decode_sequence(input_seq)
#     print("-")
#     print("Input sentence:", input_texts[seq_index])
#     print("Decoded sentence:", decoded_sentence)